In [2]:
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingLR
import os
import numpy as np
# import torch.nn.functional as F
from torch.utils.data import DataLoader
from preprocessing import fen_to_piece_maps
from tqdm import tqdm

torch.set_float32_matmul_precision('medium')

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if DEVICE == torch.device("cpu"):
    print("Using CPU, not recommended")

In [22]:
def collate_fn(batch):
    batch_fens = [example['fen'] for example in batch]
    labels = torch.tensor(
        [example['target'] for example in batch],
        dtype=torch.float32
    )

    winning_labels = []
    inputs = []

    for example in batch:
        fen = example['fen']
        target = example['target']
        
        # Parse side to move
        stm = fen.split()[1]  # 'w' or 'b'

        # Compute winning label
        if (stm == 'w' and target > 0) or (stm == 'b' and target < 0):
            winning = 1
        else:
            winning = 0

        winning_labels.append(winning)

        # Process input
        inputs.append(torch.tensor(fen_to_piece_maps(fen), dtype=torch.float32))

    inputs = torch.stack(inputs)
    winning_labels = torch.tensor(winning_labels, dtype=torch.float32)  # Make sure it's float32 for BCEWithLogitsLoss etc.

    return inputs, labels, winning_labels

In [23]:
train_dataset = load_from_disk(os.path.join(os.getcwd(), "processed_data/lichess_db_eval_10m/train"))
val_dataset = load_from_disk(os.path.join(os.getcwd(), "processed_data/lichess_db_eval_10m/validation"))
test_dataset = load_from_disk(os.path.join(os.getcwd(), "processed_data/lichess_db_eval_10m/test"))

num_training_examples = len(train_dataset)

train_dataset = train_dataset.to_iterable_dataset(num_shards=32)
val_dataset = val_dataset.to_iterable_dataset()
test_dataset = test_dataset.to_iterable_dataset()

train_dataset = train_dataset.shuffle(buffer_size=10000)
val_dataset = val_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.shuffle(buffer_size=10000)

In [ ]:
# dataset = load_from_disk(os.path.join(os.getcwd(), "processed_data/lichess_db_eval_10m"))

# # Split the dataset into train, validation, and test sets
# train_size = int(0.98 * len(dataset))
# val_size = int(0.01 * len(dataset))
# test_size = len(dataset) - train_size - val_size

# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
#     dataset, [train_size, val_size, test_size]
# )

# # Convert to iterable datasets
# train_dataset = train_dataset.dataset.to_iterable_dataset(num_shards=32)
# val_dataset = val_dataset.dataset.to_iterable_dataset()
# test_dataset = test_dataset.dataset.to_iterable_dataset()

# # Shuffle the datasets
# train_dataset = train_dataset.shuffle(buffer_size=10000)
# val_dataset = val_dataset.shuffle(buffer_size=10000)
# test_dataset = test_dataset.shuffle(buffer_size=10000)

In [24]:
train_loader = DataLoader(train_dataset, batch_size=256, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=256, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=256, collate_fn=collate_fn)

In [25]:
import torch
import torch.nn as nn

class FFN(nn.Module):
    def __init__(self, input_size, hidden_size=256, dropout_prob=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        return x

class ChessEvalMLP(nn.Module):
    def __init__(self, input_planes=17, hidden_size=256, dropout_prob=0.2):
        super().__init__()
        input_size = input_planes * 8 * 8  # because of piece maps (channels * 8 * 8)

        self.backbone = FFN(input_size, hidden_size=hidden_size, dropout_prob=dropout_prob)

        # Two heads
        self.eval_head = nn.Linear(hidden_size // 2, 1)  # For regression
        self.win_head = nn.Linear(hidden_size // 2, 1)   # For binary classification

    def forward(self, x):
        x = torch.flatten(x, 1)  # (batch_size, channels*8*8)
        features = self.backbone(x)
        eval_output = self.eval_head(features)     # Regression output (centipawns)
        win_output = self.win_head(features)       # Classification output (winning/not)

        return eval_output, win_output

In [26]:
input_tensor = torch.randn(32, 17, 8, 8)  # A batch of 32 chessboard positions

model = ChessEvalMLP()
output = model(input_tensor)

print(output[0].shape, output[1].shape)

torch.Size([32, 1]) torch.Size([32, 1])


In [27]:
NUM_EPOCHS = 3
total_iters = NUM_EPOCHS * ((10_000_000 // 256) + 1)

model = ChessEvalMLP().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=total_iters, eta_min=1e-4)
eval_criterion = nn.MSELoss()
win_criterion = nn.BCEWithLogitsLoss()

steps, train_losses, train_maes, val_losses, val_maes = [], [], [], [], []       # for tracking performance

In [28]:
best_val_loss = float('inf')
best_val_mae = float('inf')
num_iterations = 0
patience_counter = 0

PATIENCE = 15000
MIN_IMPROVEMENT = 5e-4
VAL_ITERS = 500
LOG_ITERS = 100

early_stop = False

steps = []
train_losses, train_maes, train_win_accs = [], [], []
val_losses, val_maes, val_win_accs = [], [], []

for epoch in range(1, NUM_EPOCHS + 1):
    if early_stop:
        break

    model.train()
    total_loss = 0.0
    total_mae = 0.0
    total_win_correct = 0
    total_samples = 0

    for inputs, labels, winning_labels in tqdm(train_loader, desc=f"Epoch {epoch}"):
        inputs, labels, winning_labels = inputs.to(DEVICE), labels.to(DEVICE), winning_labels.to(DEVICE)

        optimizer.zero_grad()
        eval_pred, win_pred = model(inputs)

        loss_eval = eval_criterion(eval_pred.squeeze(), labels)
        loss_win = win_criterion(win_pred.squeeze(), winning_labels)

        loss = loss_eval + loss_win

        loss.backward()
        optimizer.step()
        scheduler.step()

        batch_loss = loss.item()
        batch_mae = torch.mean(torch.abs(eval_pred.squeeze() - labels)).item()

        # Calculate batch binary accuracy
        win_pred_labels = (torch.sigmoid(win_pred.squeeze()) > 0.5).float()
        batch_win_acc = (win_pred_labels == winning_labels).float().mean().item()

        total_loss += batch_loss * inputs.size(0)
        total_mae += batch_mae * inputs.size(0)
        total_win_correct += (win_pred_labels == winning_labels).sum().item()
        total_samples += inputs.size(0)
        num_iterations += 1

        if num_iterations % LOG_ITERS == 0:
            avg_train_loss = total_loss / total_samples
            avg_train_mae = total_mae / total_samples
            avg_train_win_acc = total_win_correct / total_samples

            print(f"Step {num_iterations} — train_loss: {avg_train_loss:.4f}, train_mae: {avg_train_mae:.4f}, train_win_acc: {avg_train_win_acc:.4f}")

    # Validation every VAL_ITERS iterations
        if num_iterations % VAL_ITERS == 0:
            model.eval()
            val_loss_sum = 0.0
            val_mae_sum = 0.0
            val_win_correct = 0
            val_total_samples = 0

            with torch.no_grad():
                for val_inputs, val_labels, val_winning_labels in val_loader:
                    val_inputs, val_labels, val_winning_labels = val_inputs.to(DEVICE), val_labels.to(DEVICE), val_winning_labels.to(DEVICE)

                    val_pred, val_win_pred = model(val_inputs)

                    val_loss = eval_criterion(val_pred.squeeze(), val_labels)
                    val_mae = torch.mean(torch.abs(val_pred.squeeze() - val_labels))

                    val_loss_sum += val_loss.item() * val_inputs.size(0)
                    val_mae_sum += val_mae.item() * val_inputs.size(0)

                    val_win_preds = (torch.sigmoid(val_win_pred.squeeze()) > 0.5).float()
                    val_win_correct += (val_win_preds == val_winning_labels).sum().item()

                    val_total_samples += val_inputs.size(0)

            avg_val_loss = val_loss_sum / val_total_samples
            avg_val_mae = val_mae_sum / val_total_samples
            avg_val_win_acc = val_win_correct / val_total_samples

            avg_train_loss = total_loss / total_samples
            avg_train_mae = total_mae / total_samples
            avg_train_win_acc = total_win_correct / total_samples

            print(f"\n[Validation] Step {num_iterations} — train_loss: {avg_train_loss:.4f}, train_mae: {avg_train_mae:.4f}, train_win_acc: {avg_train_win_acc:.4f}, val_loss: {avg_val_loss:.4f}, val_mae: {avg_val_mae:.4f}, val_win_acc: {avg_val_win_acc:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

            steps.append(num_iterations)
            train_losses.append(avg_train_loss)
            train_maes.append(avg_train_mae)
            train_win_accs.append(avg_train_win_acc)

            val_losses.append(avg_val_loss)
            val_maes.append(avg_val_mae)
            val_win_accs.append(avg_val_win_acc)

            # Checkpoint best
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save(model.state_dict(), "best_chess_mlp.pth")
                print(f"Saved new best model after {num_iterations} iters")
            
            # Early stopping on validation MAE
            if avg_val_mae + MIN_IMPROVEMENT < best_val_mae:
                best_val_mae = avg_val_mae
                patience_counter = 0
                print(f"--------Validation MAE improved to {best_val_mae:.6f}--------")
            else:
                patience_counter += VAL_ITERS
                print(f"--------No significant MAE improvement for {patience_counter} iterations--------")

            if patience_counter >= PATIENCE:
                print(f"Early stopping triggered at {num_iterations} iterations (no significant MAE improvement)")
                early_stop = True
                break

            model.train()


Epoch 1: 102it [00:05, 20.42it/s]

Step 100 — train_loss: 0.8589, train_mae: 0.3063, train_win_acc: 0.5989


Epoch 1: 202it [00:10, 20.17it/s]

Step 200 — train_loss: 0.8243, train_mae: 0.2928, train_win_acc: 0.6230


Epoch 1: 301it [00:14, 20.57it/s]

...


[Validation] Step 8000 — train_loss: 0.6429, train_mae: 0.2207, train_win_acc: 0.7227, val_loss: 0.0955, val_mae: 0.1994, val_win_acc: 0.7482, LR: 0.000990
Saved new best model after 8000 iters
--------Validation MAE improved to 0.199361--------


Epoch 1: 8103it [18:49, 20.11it/s]

Step 8100 — train_loss: 0.6425, train_mae: 0.2205, train_win_acc: 0.7229


Epoch 1: 8193it [18:54,  7.22it/s]


KeyboardInterrupt: 